# Genetic algorithms: 1D example

Import some packages/modules

In [ ]:
import numpy as np
import pandas as pd
from random import random
import plotly.express as px
import plotly.graph_objects as go
from time import sleep
from IPython.display import clear_output

from fonctions import * # TODO remove by moving content in the notebook

# Set the seed of the random number generator.
# `np.random.seed(value)` is considered a legacy function,
# so let's use a `np.random.Generator`
rng: np.random.Generator = np.random.default_rng(seed=112358)

Notebook settings

In [ ]:
EXPORT_FIGURES = True

# Objective function

Define the function to minimize : $f(x) = −0.02x \times sin(0.01 x \times 2 \pi) − 4$

With $x$ an integer between $0$ and $2^8 = 255$ included.

In [ ]:
function_to_minimize = lambda x: -0.02 * x * np.sin(0.01*x*2*np.pi) - 4

# compute its value over the domain
x = np.arange(0, 2**8) # range [0, 2^8=256[ -> [0, 255]
y = function_to_minimize(x) # evaluate all values in x

# define how to plot the function
def plot_objective_function(x: np.ndarray, y: np.ndarray) -> go.Figure:
    fig = go.Figure(
        go.Scatter(x=x,y=y,mode='lines'),
        layout_xaxis_range=[0,255],
        layout_yaxis_range=[-9,0]
    )
    fig.update_layout(title_text="Function to minimize")
    return fig

# plot the function
fig = plot_objective_function(x,y)
fig.show()

if EXPORT_FIGURES:
    fig.write_image("function_to_minimize.png")

print(f"The minimum is {np.min(y):0.2f} at x={np.argmin(y)}")

# Binary representation

In order to use a genetic algorithm, we must be able to represent solutions (= individuals) as chromosomes -> alleles vector.

The simplest way to do so with integers is to use their binary representation.

But there is a catch: to enforce adjacent inputs to have adjacent binary codes, we must use the [Gray code](https://en.wikipedia.org/wiki/Gray_code).

This way, the binary representation of two successive integers will differ in only one bit.

We need to functions, `dec2gc()` to compute the Gray code of a decimal number, and `gc2dec()` the inverse.

In [ ]:
# https://www.geeksforgeeks.org/decimal-equivalent-gray-code-inverse/

def dec2gc(dec,N):
    """
    Decimal to Gray code conversion
    """
    binary = dec
    binary ^= (binary >> 1) # conversion happens here
    # convert to string with bin(), remove '0b', pad with '0's for fixed width
    binary = '{:0>{width}}'.format(bin(binary)[2:], 'b', width=N)
    # separate chars with ' ' and use this separator to get an int array
    return np.fromstring(" ".join(binary),dtype=int,sep=" ")

def gc2dec(gc):
    """
    Gray code to decimal conversion
    """
    #create a string from the array
    gc = np.array2string(gc, separator='')[1:-1]
    gc = int(gc,base=2)
    inv = 0
    while(gc):
        inv = inv ^ gc
        gc = gc >> 1
    return inv

Test the conversion functions

In [ ]:

decimal_value = np.random.randint(0,2**8) # random 8 bits integer
gray_code = dec2gc(decimal_value,8)
print(f"The Gray code (on 8 bits) of {decimal_value} is\n{gray_code}\n")
back_to_decimal = gc2dec(gray_code)
print(f"The decimal value of\n{gray_code} is {back_to_decimal}")
assert(back_to_decimal == decimal_value)

# Generation of the initial population

The population is stored as a $N ~ \text{individuals} \times 8 ~ \text{genes}$ matrix. Here $N=20$.

Option A : pre-generated random individuals

In [ ]:
population = np.array([[1,1,0,1,0,0,0,1],
                       [1,0,0,0,1,1,1,1],
                       [0,1,1,1,1,0,0,0],
                       [1,1,1,1,1,1,1,1],
                       [1,1,0,1,1,0,1,0],
                       [0,1,0,1,0,1,1,0],
                       [0,1,0,1,1,0,0,0],
                       [1,0,1,0,1,1,0,0],
                       [1,1,1,0,0,1,0,0],
                       [1,0,1,0,1,1,1,0],
                       [0,1,0,1,0,0,0,0],
                       [1,0,1,0,0,0,1,0],
                       [1,0,1,1,1,0,0,1],
                       [0,0,0,0,1,1,0,1],
                       [1,0,0,1,1,0,0,0],
                       [0,1,0,0,0,1,0,0],
                       [0,1,1,0,1,1,1,0],
                       [1,0,0,0,0,1,1,1],
                       [1,1,1,1,0,0,1,0],
                       [1,0,0,0,0,0,0,0]], dtype=int)

Option B : populate the initial population with random individuals

In [ ]:
population = rng.integers(low=0, high=2, size=(20,8))

Plot the population

In [ ]:
# define how to plot a given population
def plot_population(x,y,population) -> go.Figure:
    fig = plot_objective_function(x,y)
    x_population = np.apply_along_axis((lambda x: float(gc2dec(x))), axis=1, arr=population)
    y_population = function_to_minimize(x_population)
    fig.add_trace(go.Scatter(
        x=x_population,
        y=y_population,
        mode='markers',
        marker_color='black',
        marker_size=10
    ))
    fig.layout.update(showlegend=False) # remove legend
    return fig

# plot the initial population
fig = plot_population(x,y,population)
fig.update_layout(title_text='Initial population')
fig.show()

if EXPORT_FIGURES:
    fig.write_image("initial_population.png")

## Test `calcul_scores_population`, `afficher_population_et_scores` et `afficher_stats_scores`

In [ ]:
scores = calcul_scores_population(population)
afficher_population_et_scores(population,scores)
afficher_stats_scores(scores,0)

## Test selection

In [ ]:
a = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])
scores = np.array([[0.5],
                   [1.2],
                   [0.1],
                   [5.2]])
afficher_population_et_scores(a,scores)

In [ ]:
individus_selectiones = selection(a,scores)
print(individus_selectiones)

## Test melange_parents

In [ ]:
a_melange = melange_parents(a)
print(a_melange)

## Test `croisement`

In [ ]:
p1 = np.array([0,0,0,1,1,0,1,1])
p2 = np.array([1,0,1,1,0,0,1,0])
(e1,e2,point_croisement) = croisement(p1,p2)
print("Parents:")
afficher_genes_individu(p1)
print()
afficher_genes_individu(p2)
print()
print("Enfants:")
afficher_genes_individu(e1)
print()
afficher_genes_individu(e2)
print()
#print("point de croisement = ",point_croisement)
print(" "*(point_croisement)+"][")
print("(point_croisement="+str(point_croisement)+")")

## Test `index_des_meilleurs`

In [ ]:
scores2 = np.array([[0.5],#0
                    [1.2],#1
                    [0.1],#2
                    [5.2],#3
                    [0.7],#4
                    [0.9],#5
                    [4.0],#6
                    [3.1]])#7
meilleurs = index_des_meilleurs(scores2,4)
pires = index_des_pires(scores2,5)
print("meilleurs=",meilleurs)
print("pires=",pires)

In [ ]:
proba_croisement = 0.8
proba_mutation = 0.08
max_generations = 100
nb_individus_remplaces = 5
afficher_etapes = True
afficher_evolution_du_score = True
scores_moyens = np.zeros((1,max_generations+1))
meilleurs_scores = np.zeros((1,max_generations+1))
boucle_optimisation(population,proba_croisement,proba_mutation,max_generations,nb_individus_remplaces,afficher_etapes,afficher_evolution_du_score,scores_moyens,meilleurs_scores)

Then to obain a GIF of the evolving population, with [Gifski](https://github.com/ImageOptim/gifski/) :
```bash
gifski -o anim.gif generation_*.png --fps 4
```